In [163]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import csv
import numpy as np
import random
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [214]:
df = pd.read_csv('train.csv')

df_train_y = df['Survived'].values
df_train_x = df.drop(['Survived'], axis=1)
df_test_x = pd.read_csv('test.csv')

test_passenger_ids = df_test_x['PassengerId']
test_passenger_ids = np.reshape(test_passenger_ids.values, (df_test_x.shape[0], 1))

labels = df_train_x.columns.values

# Remove unused Columns

In [215]:
def remove_unused_cols(dataset, labels):
    unused_cols = ["PassengerId", "Name", "Cabin", "Embarked", "Ticket"]
    
    for col_name in unused_cols:
        labels = labels[labels != col_name]
        dataset = dataset.drop([col_name], axis=1)
       
    
    return dataset, labels

df_train_x, labels = remove_unused_cols(df_train_x, labels)
df_test_x, _ = remove_unused_cols(df_test_x, labels)

# Preprocessing

**Fill NaN Values for**
- Age

**Apply One Hot Encoding on**
- Sex

In [216]:
# Should we check for sibling/parents and use this as a criteria for the random value?
def fill_missing_age_fields(dataset):
    dataset["Age"].fillna(random.choice(dataset[dataset["Age"] != np.nan]["Age"]), inplace =True)
    
fill_missing_age_fields(df_train_x)
fill_missing_age_fields(df_test_x)

In [217]:
def one_hot_encode_column(dataset, col_names):
    for col_name in col_names:
        col = dataset.loc[:, col_name]
        label_encoder = LabelEncoder()
        integer_encoded = label_encoder.fit_transform(col.values)
        onehot_encoder = OneHotEncoder(sparse=False)
        integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
        onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

        dataset = dataset.drop([col_name], axis=1)

        for i in range(0, onehot_encoded.shape[1]):
            new_col_name = "{0}_{1}".format(col_name, str(i))
            dataset[new_col_name] = onehot_encoded[:, i]
    
    return dataset

columns_to_one_hot_encode = ['Pclass', 'Sex']

df_train_x = one_hot_encode_column(df_train_x, columns_to_one_hot_encode)
df_test_x = one_hot_encode_column(df_test_x, columns_to_one_hot_encode)

D:\Programs\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
D:\Programs\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a L

# Standardize Data

In [218]:
scaler = StandardScaler().fit(df_train_x)
train_x = scaler.transform(df_train_x)
test_x = scaler.transform(df_test_x)

D:\Programs\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


# Apply Magic

**Todo**:
- k-Fold Cross Validation?

In [219]:
# Define binary classification model
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=train_x.shape[1]))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [220]:
# Train the model, iterating on the data in batches of 32 samples
model.fit(train_x, train_y, epochs=20, batch_size=32)

Epoch 1/20
891/891 [==============================] - 0s 504us/step - loss: 0.6409 - acc: 0.6049
Epoch 2/20
891/891 [==============================] - 0s 100us/step - loss: 0.5447 - acc: 0.8002
Epoch 3/20
891/891 [==============================] - 0s 102us/step - loss: 0.4913 - acc: 0.8047
Epoch 4/20
891/891 [==============================] - 0s 102us/step - loss: 0.4604 - acc: 0.8092
Epoch 5/20
891/891 [==============================] - 0s 102us/step - loss: 0.4448 - acc: 0.8103
Epoch 6/20
891/891 [==============================] - 0s 106us/step - loss: 0.4366 - acc: 0.8126
Epoch 7/20
891/891 [==============================] - 0s 107us/step - loss: 0.4313 - acc: 0.8092
Epoch 8/20
891/891 [==============================] - 0s 101us/step - loss: 0.4276 - acc: 0.8103
Epoch 9/20
891/891 [==============================] - 0s 101us/step - loss: 0.4245 - acc: 0.8148
Epoch 10/20
891/891 [==============================] - 0s 104us/step - loss: 0.4217 - acc: 0.8171
Epoch 11/20
891/891 [========

In [221]:
# Run the model against the test data
predict_y = model.predict(test_x)
predict_y = np.around(predict_y)
predict_y = predict_y.astype(np.integer)

In [226]:
# Write our predictions to a csv file
csv_predict = np.concatenate((test_passenger_ids, predict_y), axis=1)
csv_predict = np.concatenate((np.reshape(["PassengerId", "Survived"], (1, 2)), csv_predict))
with open('prediction.csv', 'w', newline='') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(csv_predict)
csvFile.close()